In [1]:
import numpy as np

class Node:
    def __init__(self,inbound_nodes=[]):
        self.inbound_nodes = inbound_nodes
        self.outbound_nodes = []
        self.value = None
        for n in self.inbound_nodes:
            n.outbound_nodes.append(self)
            
    def forward(self):
        raise NotImplemented

In [2]:
class Input(Node):
    def __init__(self):
        Node.__init__(self)
        
    def forward(self,value=None):
        if value is not None:
            self.value = value

In [3]:
class Add(Node):
    def __init__(self,*inputs):
        Node.__init__(self,inputs)
        
    def forward(self):
        self.value = sum([n.value for n in self.inbound_nodes])

In [9]:
class Linear(Node):
    def __init__(self,inputs,weights,bias):
        Node.__init__(self,[inputs,weights,bias])
        
    def forward(self):
        np_inputs = self.inbound_nodes[0].value
        np_weights = self.inbound_nodes[1].value
        bias = self.inbound_nodes[2].value
        self.value = np_inputs.dot(np_weights) + bias

In [30]:
class MSE(Node):
    def __init__(self,y,a):
        Node.__init__(self,[y,a])
        
    def forward(self):
        actual_output = self.inbound_nodes[0].value.reshape(-1,1)
        prediction = self.inbound_nodes[1].value.reshape(-1,1)
        num_output = np.shape(prediction)[0]
        squared_error = sum(np.square(actual_output - prediction))
        self.value = squared_error/float(num_output)

In [22]:
class Sigmoid(Node):
    def __init__(self,x):
        Node.__init__(self,[x])
        
    def _sigmoid(self,x):
        return 1/(1 + np.exp(-x))
        
    def forward(self):
        self.value = self._sigmoid(self.inbound_nodes[0].value)

In [19]:
def topological_sort(feed_dict):
    G = {}
    input_nodes = [n for n in feed_dict.keys()]
    nodes = [n for n in input_nodes]
    
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in':set(),'out':set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in':set(),'out':set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)
    
    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()
        
        if isinstance(n,Input):
            n.value = feed_dict[n]
        
        L.append(n)
        for m in n.outbound_nodes:
            G[m]['in'].remove(n)
            G[n]['out'].remove(m)
            
            if len(G[m]['in']) == 0:
                S.add(m)
    
    return L
            

In [20]:
def forward_pass(output_node,sorted_nodes):
    for node in sorted_nodes:
        node.forward()
    
    return output_node.value

In [6]:
x,y,z = Input(),Input(),Input()

f = Add(x,y,z)

feed_dict = {x:1,y:2,z:3}

sorted_nodes = topological_sort(feed_dict)

output = forward_pass(f,sorted_nodes)

print(output)

6


In [28]:
inputs,weights,bias = Input(),Input(),Input()

f = Linear(inputs,weights,bias)

g = Sigmoid(f)

feed_dict = {
    inputs:np.array([[-1,-2],[-1,-2]]),
    weights:np.array([[2,-3],[2,-3]]),
    bias:np.array([-3,-5])
}

sorted_nodes = topological_sort(feed_dict)

output = forward_pass(g,sorted_nodes)

print(output)

[[  1.23394576e-04   9.82013790e-01]
 [  1.23394576e-04   9.82013790e-01]]


In [32]:
y,a = Input(),Input()
cost = MSE(y,a)

y_ = np.array([1, 2, 3])
a_ = np.array([4.5, 5, 10])

feed_dict = {y: y_, a: a_}
graph = topological_sort(feed_dict)
# forward pass
forward_pass(cost,graph)


array([ 23.41666667])